# PZQ resistance: genome-wide association study


## Aim

This notebook describes the different analyses regarding the mapping of the quantitative trait locus (QTL) associated to praziquantel (PZQ) resistance. We performed a genome-wide association study on a population of *Schistosoma mansoni* (SmLE-PZQ-R) which is polymorphic for PZQ resistance ([Couto *et al.*](https://doi.org/10.1590/s0074-02762011000200006)). The population was treated with a dose of 24 µg/mL of PZQ and survival of the worms was objectively assessed using a lactate assay. Pools of the 20% lowest and 20% higher lactacte producer were made. The GWAS was conducted with two independent biological replicates and each library was sequenced in triplicate for assessing allele frequencies. We compared allele frequencies between pools in order to identify the genomic locus involved in PZQ resistance.

After identifying the QTL related to resistance, we performed a marker assisted selection to create two populations: an enriched population in PZQ resistance allele (SmLE-PZQ-ER) and an enriched population in PZQ sensitive allele (SmLE-PZQ-ES). We confirmed the phenotype by assessing PZQ response using a lactate assay and confirmed that the desired locus was the only one differentiating the two populations.


## Environment and data

Creating a conda environment improves reproducibility by installing specific versions of the programs used.

In [ ]:
conda env create -f .env/env.yml

The cell below must be run each time a new Jupyter session is run.

In [ ]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate PZQ-R_DNA

# Remove potential variable interferences
export PERL5LIB=""
export PYTHONNOUSERSITE=1

The cell below must be run only once at the time of the environment creation.

In [ ]:
# Installing needed R packages
Rscript ".env/R package dependencies.R"

### Phenotypic data

The phenotypic data for the GWAS and selected populations must be downloaded from [Zenodo](https://doi.org/10.5281/zenodo.5209299). If further information is needed regarding the data, detailed readmes are present in each folder.

In [ ]:
# Data directory
pdir="data/phenotypes"
[[ ! -d "$pdir" ]] && mkdir -p "$pdir"

In [ ]:
# Download files
# Download the data from Zenodo
wget -P "$pdir" "https://zenodo.org/record/5209300/files/Phenotype%2C%20genotype%20and%20fitness%20data.tar.gz"

# Extract files
tar xzvf "$pdir/Phenotype, genotype and fitness data.tar.gz" -C "$pdir"

# Remove archive
rm "$pdir/Phenotype, genotype and fitness data.tar.gz"

### Sequencing data

This step downloads the fastq files of the different samples from the SRA repository.

#### GWAS

In [ ]:
# Data directory
ldir="data/libraries/1-GWAS/"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

In [ ]:
# Bioproject
bioproject=PRJNA699326

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "SampleName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)
flk=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "download_path" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename, run and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    lnk=$(cut -d "," -f $flk <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir/$fln/" ]] && mkdir -p "$ldir/$fln/"
    retry=0
    
    while [[ $retry -lt 2 ]]
    do
        # Download sra file
        wget -q -c -O "$ldir/$fln/$run" "$lnk"
        # Check integrity
        vdb-validate -q "$ldir/$fln/$run" &> /dev/null
        [[ $? -ne 0 ]] && ((retry++)) || break
    done
    
    # If max download attempt reached, issue message and move to the next
    [[ $retry -eq 2 ]] && echo "$run: dowloading problem" >> "$ldir/download_issue" && contine
    
    # Convert sra into fastq
    fastq-dump -O "$ldir/$fln/" --split-files "$ldir/$fln/$run"
    rm "$ldir/$fln/$run"
    
    # Rename file with more meaningful name
    mv "$ldir/$fln/${run}_1.fastq" "$ldir/$fln/${fln}_R1.fastq"
    mv "$ldir/$fln/${run}_2.fastq" "$ldir/$fln/${fln}_R2.fastq"
    
done < <(tail -n +2 runinfo | sed "/^$/d")

# Compress files
pigz "$ldir/"*/*

rm runinfo*

#### Selected SmLE-PZQ-ER and SmLE-PZQ-ES populations

In [ ]:
# Data directory
ldir="data/libraries/2-ER-ES_populations/"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

In [ ]:
# Bioproject
bioproject=PRJNA701978

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "SampleName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)
flk=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "download_path" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename, run and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    lnk=$(cut -d "," -f $flk <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir/$fln/" ]] && mkdir -p "$ldir/$fln/"
    retry=0
    
    while [[ $retry -lt 2 ]]
    do
        # Download sra file
        wget -q -c -O "$ldir/$fln/$run" "$lnk"
        # Check integrity
        vdb-validate -q "$ldir/$fln/$run" &> /dev/null
        [[ $? -ne 0 ]] && ((retry++)) || break
    done
    
    # If max download attempt reached, issue message and move to the next
    [[ $retry -eq 2 ]] && echo "$run: dowloading problem" >> "$ldir/download_issue" && contine
    
    # Convert sra into fastq
    fastq-dump -O "$ldir/$fln/" --split-files "$ldir/$fln/$run"
    rm "$ldir/$fln/$run"
    
    # Rename file with more meaningful name
    mv "$ldir/$fln/${run}_1.fastq" "$ldir/$fln/${fln}_R1.fastq"
    mv "$ldir/$fln/${run}_2.fastq" "$ldir/$fln/${fln}_R2.fastq"
    
done < <(tail -n +2 runinfo | sed "/^$/d")

# Compress files
pigz "$ldir/"*/*

rm runinfo*

### Genome data

The genome data is downloaded from the [WormBase ParaSite](https://parasite.wormbase.org). We use the data from the version 14 (WBPS14). The data is then indexed by the different tools used in the downstream pipeline.

In [ ]:
gdir="data/genome"
[[ ! -d "$gdir" ]] && mkdir -p "$gdir"

# Download and unzip data
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz"

In [ ]:
# Preparing indices
bwa index "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
samtools faidx "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
gatk CreateSequenceDictionary -R "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"

### snpEff database

To annotate the functional consequences of the variants in the VCF file, we use the snpEff tool. Because the reference genome is recent, we have to build a snpEff database in order to annotate the variants. 

In [ ]:
# Create snpEff folder
sedir="$gdir/snpEff/"
mkdir "$sedir"

# Download the S. mansoni genome annotation
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz

pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz"

# Convert gff to gtf to make it easier for snpEff
gffread "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" -T -o "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf"

# Custom snpEff config file
cp "$CONDA_PREFIX"/share/snpeff-4.3.1t-4/snpEff.config "$sedir"

## Identify mitochondrial chromosome
mito=$(grep -i ">.*mito.*" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" | sed -r "s/>(.*) .*/\1/")

##  Add schistosome genome and specify mitochondrial chromosome
sed -i -r "s|data.dir = ./data/|data.dir = .| ; s|(# Ebola virus)|# S. mansoni\nsmansoni.genome: Schistosoma_mansoni\n\tsmansoni.$mito.codonTable : Trematode_Mitochondrial\n\n\1|" "$sedir"/snpEff.config

# Create the mandatory folder and link to the needed files
mkdir "$sedir"/genomes
ln -s "$PWD/$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" "$PWD/$sedir/genomes/smansoni.fa"

mkdir "$sedir"/smansoni
ln -s "$PWD/$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf" "$PWD/$sedir/smansoni/genes.gtf"

# Generate the database
snpEff build -v -c "$sedir/snpEff.config" smansoni

### Known variants

List of variants from the SmLE x SmHR crosses that segregated in a Mendelian fashion in F1 ([Valentim *et al.* 2013](https://doi.org/10.1126/science.1243106)). This list was generated from alignments using the v5 genome. This has been lifted over to the latest genome using the [flo pipeline](https://github.com/wurmlab/flo/tree/727f10b2b1c57a0514835d302d7f6345d3a34ffb).

In [ ]:
# Index file
gatk IndexFeatureFile -I "$gdir/sm_dbSNP_v7.vcf"

## Genome-wide assocation study

### Phenotypes

#### PZQ response of SmLE and SmLE-PZQ-R

We tested PZQ response of SmLE and SmLE-PZQ-R using several doses of PZQ. We assessed mortality based on worm movement. We noticed that 35% of the worms from the SmLE-PZQ-R population always recovered from the highest treatment, suggesting fully resistant worms.

In [ ]:
Rscript scripts/Fig1_IC_curves_script_LE_PZQR.R

#### Validation of lactate assay as phenotyping tool 

Because visual assessement of "mortality" through movement is not high throughtput and there is no working software to quantify single worm movement in schistosome from video recordings, we decided to quantify L-lactate production, a surrogate measure of respiration, to efficiently quantify the worm recovery from PZQ treatment. We validate the method by comparing the lactate production with visual assessement.

In [ ]:
Rscript scripts/FigS1_Single_worm_lactate_data_sorted.R

#### Worm selection for the GWAS

We treated single worms with a dose of 24 µg/mL and quantify the lactate production of each. We took the 20% lowest and highest lactate producers from the distribution to build the sensitive (contracted) and resistant (recovered) worm pools, respectively. The experiment was done in duplicate. We then extracted DNA from worm pools, prepared the libraries and sent it for sequencing.

**Note:** this script generates the list of worms that were selected. However, it should be run after doing the sequencing data and variant calling because it generates the figure 2 of the manuscript which requires variant data.

In [ ]:
Rscript scripts/Fig2_PZQ_resistance_association_study.R

### Sequencing data processing

Sequencing data are processed using a snakemake pipeline. The pipeline performs the following steps:
* alignment of each library against the reference genome
* marking duplicates
* base quality recalibration
* generating alignment statistics
* calling variants (gVCF) for each library
* combining gVCF of all libraries
* genotyping all samples per contig
* merging all the contigs to generate final VCF file

The VCF file produced is then be used to identify possible QTLs link to the PZQ resistance.

In [ ]:
# Status directory
statdir=status
[[ ! -d "$statdir" ]] && mkdir -p "$statdir"

In [ ]:
snakemake --snakefile GWAS.smk --cluster "qsub -V -cwd -o $statdir -j y -r y -l mem_free=50G -S /bin/bash" --jobs 384 -w 120

### Variant annotation

Functional consequences of variants is annotated to the VCF file using snpEff.

In [ ]:
# Variant calling directory
cdir="data/calling"
[[ ! -d "$cdir" ]] && mkdir -p "$cdir"

myvcf=$(grep -m 1 VCF_PFX GWAS.smk | cut -d "\"" -f 2)

snpEff -v -c "$sedir/snpEff.config" smansoni "$cdir/$myvcf.vcf.gz" | bgzip -c -@ $(nproc) > "$cdir/${myvcf}_snpEff.vcf.gz"

# Index the VCF file
tabix "$cdir/${myvcf}_snpEff.vcf.gz"

# Move file in the right folder
mv -t "$cdir/" snpEff_summary.* snpEff_genes.txt

### QTL analysis

The analysis relies on several steps:
* **SNP filtration**: bi-allelic sites only, minimum read depth (DP) of 10, and maximum standard deviation between triplicated libraries of 10. Filtering with read depth and standard deviation is done after Z score calculation. We did not use genotype quality as a filtering parameter because this is not relevant for pool genotyping.
* **Estimation of the genotype data**: a genotype at each variable site is estimated for each pool based on allele frequencies.
* **Allele frequency polarization**: allele frequencies are polarized (0 = fixed in resistant pool, 1 = fixed in sensitive). These polarized allele frequencies are then subtracted to determine origin of alleles explaining the difference in Z score.
* **Computation of the Z scores**: Z scores are computed for each variant based on allele frequencies between recovered and contracted pools. Then Z scores are combined between replicated experiment at common variable sites between pools.
* **QTL boundaries**: we determine the boundaries of each QTL passing the Bonferroni correction (variants are considered in the same QTL if they are at max 500 kb apart).
* **Mutation and gene tables**: for each identified QTL, a table of mutations and a summary table of genes are generated.
* **Figures**: figures are eventually generated regarding quality filtering, QTL mapping, allele polarization and origin of allele (allele frequency subtraction).

**Note**: The analysis will take at least half an hour on powerful servers (192 1.9 GHz CPUs / 1 TB of RAM).

In [ ]:
Rscript scripts/PZQ-R_GWAS_X-QTL.R

The analysis shows the presence of two QTLs on chr. 2 and chr. 3.

We confirmed our findings using the G' statistics, an alternative method to the Z score. We used the R package [QTLseqr](https://doi.org/10.3835/plantgenome2018.01.0006). Results are highly similar.

In [ ]:
Rscript scripts/QTLseqr_analysis.R

### Copy number variation

In order to identify copy number variation (CNV) into the QTL, we generate a coverage file of the chromosome 3 for each library. This will produce a table with read depth at each position of the chromosome. Only read depth greater than 0 will be retained.

In [ ]:
for i in $(find data/libraries/1-GWAS/ -name "*_sorted_MD_recal.bam" | sort)
do
    sample=$(basename $(dirname "$i"))
    output="$(dirname "$i")/${sample}_3_QTL.cov"
    echo "samtools view -u $i SM_V7_3 | bedtools genomecov -ibam - -d | grep -w SM_V7_3 | awk '\$3 > 0 {print}' > $output" | qsub -V -cwd -o "$statdir" -j y -r y -pe smp 12
done

The table is then analyzed using the R package [cn.mops](https://doi.org/doi:10.18129/B9.bioc.cn.mops).

In [ ]:
Rscript scripts/PZQ-R_GWAS_CNV.R

In [ ]:
Rscript scripts/PZQ-R_GWAS_CNV_graph_RD.R

This revealed the presence of two CNVs within the QTL:
* The first very close to the end of the Sm.TRPM$_{PZQ}$ gene
* The second few hundreds kb away, close to a transcription factor

### Mode of inheritance of PZQ response

To confirm these associations and determine whether the loci underlying PZQ response are inherited in a dominant, co-dominant or recessive manner, we compared genotype and PZQ-response phenotype in individual worms. We exposed individual worms to 24 μg/mL PZQ and measured lactate production 48 hours after exposure. We then genotyped the worms at SNPs at the peaks of the chr. 2 and chr. 3 QTLs. We also examined copy number of one of the 150 kb deletion observed on chr. 3 using qPCR.

In [ ]:
Rscript scripts/Fig3_chr2_chr3_CNV_lactate_script.R

We observed significant differences in lactate production among genotypes. Both the copy number variant and the SNP assayed in Sm.TRPM$_{PZQ}$ revealed that the causative gene in the chr. 3 QTL showed recessive inheritance. Homozygous parasites carrying two copies of the Sm.TRPM$_{PZQ}$-741987C allele (or two copies of the deletion) recovered from PZQ treatment, while the heterozygote and other homozygotes failed to recover from treatment. For the chr. 2 QTL, we did not see a significant association between parasite genotype and PZQ-R phenotype nor with lactate production before PZQ treatment. 


### Sm.TRPM$_{PZQ}$ mutations

In order to identify the possible mutations responsible for PZQ resistance, we listed variants in the Sm.TRPM$_{PZQ}$ (Smp_246790) gene, their potential impact on the protein from the longest isoform (isoform 5) and the allele frequencies.

In [ ]:
# List of libraries
for i in $(find data/libraries/1-GWAS -maxdepth 1 -printf '%P\n' | sed "/^$/d")
do
    echo -e "$i\t$(cut -d "_" -f 3 <<< "$i")"
done | sort -k2 > data/pop_GWAS

In [ ]:
# VCF filename
myvcf="$cdir/$(grep -m 1 VCF_PFX GWAS.smk | cut -d "\"" -f 2).vcf.gz"

# Working directory
mutdir="results/3-Mutations in SmTRP/1-GWAS"
[[ ! -d "$mutdir" ]] && mkdir -p "$mutdir"

scripts/func-table-mut.sh -v "$myvcf" \
    -r "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"\
    -g "$gdir/Smp_246790.5.gff" \
    -p data/pop_GWAS \
    -o "$mutdir/Smp_246790.tsv"

## Selected SmLE-PZQ-ER and SmLE-PZQ-ES populations

We selected a marker associated with resistance/sensitivity to PZQ in the QTL region. We then performed mono-miracidial infection of snails to generate single worm genotypes. We used an a PCR-RFLP assay in order to analyze each parasite genotype to identify carrier of the marker associated with resistance or sensitivity. We then sexed the parasite genotype and used 32 males and 32 females parasite genotypes to generate either a populations enriched in resistant allele (ER) or in sensitive allele (ES).

### Phenotypes

In order to confirm PZQ resistance or sensitivity of each population, we tested single worms from ER and ES populations with several doses (similarly to the test of SmLE-PZQ-R). Results showed that the ER and ES populations differ by more than 377-fold in PZQ response.

In [ ]:
Rscript scripts/Fig4_IC_curves_script_ES_ER.R

### Sequencing data processing

In order to ensure that the marker-assisted selection worked as intended, we sequenced pooled worms from ER and ES populations. We run a similar snakemake pipeline as for the GWAS.

In [ ]:
snakemake --snakefile Selected_populations.smk --cluster "qsub -V -cwd -o $statdir -j y -r y -l mem_free=50G -S /bin/bash" --jobs 384 -w 120

### Genomic analysis

We analyze the genetic differences between the two new ER and ES populations. We expect them to be genetically identical except at the locus selected. The analysis perform is identical to the one used to identify the QTLs.

**Note**: The analysis will take at least half an hour on powerful servers (192 1.9 GHz CPUs / 1 TB of RAM).

In [ ]:
Rscript scripts/PZQ-R_ER-ES_X-QTL.R

### Copy number variation

We perform the same analysis as for the GWAS in order to confirm the presence of CNVs and their association with the resistant phenotype.

In [ ]:
for i in $(find data/libraries/2-ER-ES_populations/ -name "*_sorted_MD_recal.bam" | sort)
do
    sample=$(basename $(dirname "$i"))
    output="$(dirname "$i")/${sample}_3_QTL.cov"
    echo "samtools view -u $i SM_V7_3 | bedtools genomecov -ibam - -d | grep -w SM_V7_3 | awk '\$3 > 0 {print}' > $output" | qsub -V -cwd -o "$statdir" -j y -r y -pe smp 12
done

The table is then analyzed using the R package [cn.mops](https://doi.org/doi:10.18129/B9.bioc.cn.mops) as previously. This confirmed the presence of the CNV between ER and ES, ER showing a complete deletion at the two CNV sites.

In [ ]:
Rscript scripts/PZQ-R_ER-ES_CNV_graph_RD.R

### Sm.TRPM$_{PZQ}$ mutations

We performe mutation analysis with SmLE-PZQ-ER and ES similar to the one done with SmLE-PZQ-R.

In [ ]:
for i in $(find data/libraries/2-ER-ES_populations -maxdepth 1 -printf '%P\n' | sed "/^$/d")
do
    echo -e "$i\t$(cut -d "_" -f 1 <<< "$i")"
done | sort -k2 > data/pop_ER-ES

In [ ]:
# VCF filename
myvcf="$cdir/$(grep -m 1 VCF_PFX Selected_populations.smk | cut -d "\"" -f 2).vcf.gz"

# Working directory
mutdir="results/3-Mutations in SmTRP/2-ER-ES_populations"
[[ ! -d "$mutdir" ]] && mkdir -p "$mutdir"

scripts/func-table-mut.sh -v "$myvcf" \
    -r "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"\
    -g "$gdir/Smp_246790.5.gff" \
    -p data/pop_ER-ES \
    -o "$mutdir/Smp_246790.5.tsv"

### Fitness of SmLE-PZQ-ER and SmLE-PZQ-ES parasite populations

Both laboratory selected and field isolated *S. mansoni* showing PZQ-R have been difficult to maintain in the laboratory: the PZQ-R trait has been rapidly lost consistent with strong selection against this trait. It has been suggested that PZQ-R carries a fitness cost that will slow spread of this trait in the field under PZQ pressure. We measured several components of parasite fitness in SmLE-PZQ-ES and SmLE-PZQ-ER parasites during laboratory passage of purified parasite lines.

In [ ]:
Rscript scripts/FigS3_fitness_ES_ER.R

We found no significant differences in infectivity to snails, snail survival, or infectivity to hamsters.

### *In vivo* efficacy of PZQ against SmLE-PZQ-ES and SmLE-PZQ-ER parasites

To determine the relationship between *in vitro* PZQ-R measured in 96-well plates, and *in vivo* resistance, we treated mice infected with either SmLE-PZQ-ER or SmLE-PZQ-ES parasites populations. Mice were treated with 120 mg/kg of PZQ at 40 days post infection. 

In [ ]:
Rscript scripts/FigS7_Invivo_ES_ER.R

We observed no significant reduction in worm burden in SmLE-PZQ-ER parasites when comparing PZQ-treated and control (DMSO) treated animals (Wilcoxon test, p = 0.393A). In contrast, we recovered significantly lower numbers of worms from PZQ-treated versus untreated mice infected with the SmLE-PZQ-ES parasite population (Wilcoxon test, p = 0.008). The percent reduction observed was significantly different between the SmLE-PZQ-ES and SmLE-PZQ-ER parasites (Wilcoxon test, p = 0.0129). Interestingly, we observed a large reduction in numbers of female worms recovered from PZQ-treated SmLE-PZQ-ES parasites relative to untreated animals (Wilcoxon test, p = 0.008), while for male worms this did not reach significance (Wilcoxon test, p = 0.089). We saw no impact of PZQ-treatment for either female or male worms in mice infected with SmLE-PZQ-ER. These results show that in vivo PZQ response in treated mice differs between SmLE-PZQ-ES and SmLE-PZQ-ER parasites. These data also suggest that the extended paralysis of male SmLE-PZQ-ES worms under PZQ treatment may reduce their ability to maintain female worms in copula.

### Chemical blockers and activators of Sm.TRPM$_{PZQ}$ modulate PZQ response

Sm.TRPM$_{PZQ}$ emerges as a strong candidate gene to explain variation in PZQ response, but validation is required. We were unsuccessful in knocking down expression of Sm.TRPM$_{PZQ}$ using either siRNA or dsRNA, possibly because this gene is expressed mainly in neural tissue. We therefore used two chemical modulators of Sm.TRPM$_{PZQ}$ activity – an agonist (AG1) and an antagonist (ANT1). We exposed SmLE-PZQ-ER and ES worns to these modulator with or without PZQ. 

In [ ]:
Rscript scripts/Fig6_TRP_blocker_activator_script.R

Addition of the blocker (ANT1) allowed SmLE-PZQ-ES worms to recover from PZQ treatment, while the activator (AG1) rendered SmLE-PZQ-ER worms sensitive to PZQ treatment in a dose dependent manner. These results are consistent with a role for Sm.TRPM$_{PZQ}$ in determining variation in PZQ response.